# Homework 1

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2018 term"

This homework covers material from the unit on distributed representations. The primary goal is to explore some new techniques for building and assessing VSMs. The code you write as part of the assignment should be useful for research involving vector representations as well.

Like all homeworks, this should be submitted via Canvas. All you have to do is paste in your answers (which are all numerical values) and include the SUNetIds of anyone you worked with. Here's a direct link to the homework form:

https://canvas.stanford.edu/courses/83399/quizzes/50268

__Contents__

0. [Questions 1–2: Dice distance [2 points]](#Questions-1–2:-Dice-distance-[2-points])
0. [Question 3: t-test reweighting [2 points]](#Question-3:-t-test-reweighting-[2-points])
0. [Questions 4–6: Reweighting and co-occurrence frequency [3 points]](#Questions-4–6:-Reweighting-and-co-occurrence-frequency-[3-points])
0. [Question 7: Meeting the GloVe objective [1 point]](#Question-7:-Meeting-the-GloVe-objective-[1-point])
0. [Question 8: Expressive eloooongation [2 points]](#Question-8:-Expressive-eloooongation-[2-points])

In [2]:
import numpy as np
import os
import pandas as pd
from mittens import GloVe
from scipy.stats import pearsonr
import vsm

## Questions 1–2: Dice distance [2 points]

First, implement [Dice distance](https://en.wikipedia.org/wiki/Sørensen–Dice_coefficient) for real-valued vectors of dimension $n$, as

$$\textbf{dice}(u, v) = 1 - \frac{
    2 \sum_{i=1}^{n}\min(u_{i}, v_{i})
}{
    \sum_{i=1}^{n} u_{i} + v_{i}
}$$

(You can use `vsm.matching` for part of this.)

Second, you might want to test your implementation. Here's a simple function for that:

In [3]:
def test_dice_implementation(func):
    """`func` should be an implementation of `dice` as defined above."""
    X = np.array([
        [  4.,   4.,   2.,   0.],
        [  4.,  61.,   8.,  18.],
        [  2.,   8.,  10.,   0.],
        [  0.,  18.,   0.,   5.]]) 
    assert func(X[0], X[1]).round(5) == 0.80198
    assert func(X[1], X[2]).round(5) == 0.67568

In [7]:
def dice_distance(u, v):
    min_array = (u >= v) * v + (v > u) * u
    return 1. - 2*np.sum(min_array)/np.sum(u+v)

In [8]:
test_dice_implementation(dice_distance)

Third, use your implementation to measure the distance between A and B and between B and C in the toy `ABC` matrix we used in the first VSM notebook, repeated here for convenience.

In [10]:
ABC = pd.DataFrame([
    [ 2.0,  4.0], 
    [10.0, 15.0], 
    [14.0, 10.0]],
    index=['A', 'B', 'C'],
    columns=['x', 'y']) 

ABC

,x,y
A,2.0,4.0
B,10.0,15.0
C,14.0,10.0


In [13]:
def abc_comparisons(df, distfunc):
    for a, b in (('A', 'B'), ('B', 'C')):
        dist = distfunc(df.loc[a], df.loc[b])
        print('{0:}({1:}, {2:}) = {3:7.02f}'.format(
            distfunc.__name__, a, b, dist))

In [17]:
abc_comparisons(ABC, dice_distance)

dice_distance(A, B) =    0.61
dice_distance(B, C) =    0.18


__To submit:__

1. Dice distance between A and B.
2. Dice distance between B and C.

(The real question, which these values answer, is whether this measure place A and B close together relative to B and C – our goal for that example.) 

## Question 3: t-test reweighting [2 points]

The t-test statistic can be thought of as a reweighting scheme. For a count matrix $X$, row index $i$, and column index $j$:

$$\textbf{ttest}(X, i, j) = 
\frac{
    P(X, i, j) - \big(P(X, i, *)P(X, *, j)\big)
}{
\sqrt{(P(X, i, *)P(X, *, j))}
}$$

where $P(X, i, j)$ is $X_{ij}$ divided by the total values in $X$, $P(X, i, *)$ is the sum of the values in row $i$ of $X$ divided by the total values in $X$, and $P(X, *, j)$ is the sum of the values in column $j$ of $X$ divided by the total values in $X$.

First, implement this reweighting scheme.

Second, test your implementation:

In [97]:
def ttest_reweight(X):
    X = X / np.sum(np.array(X))
    #print(X)
    XA = np.array(X)
    col_marginals = np.sum(XA, axis = 0, keepdims=True)
    #print(col_marginals)
    row_marginals = np.sum(XA, axis = 1, keepdims=True)
    #print(row_marginals)
    X_ind = col_marginals * row_marginals
    #print(X_ind)
    return (X - X_ind) / np.sqrt(X_ind)

In [93]:
X = pd.DataFrame((np.arange(12)).reshape((4,3)))  
#print(X)
print(ttest_reweight(X))

          0         1         2
0  0.000000  0.015152  0.030303
1  0.045455  0.060606  0.075758
2  0.090909  0.106061  0.121212
3  0.136364  0.151515  0.166667
[[0.27272727 0.33333333 0.39393939]]
[[0.04545455]
 [0.18181818]
 [0.31818182]
 [0.45454545]]
[[0.01239669 0.01515152 0.01790634]
 [0.04958678 0.06060606 0.07162534]
 [0.08677686 0.10606061 0.12534435]
 [0.12396694 0.15151515 0.17906336]]
          0             1         2
0 -0.111340 -1.409296e-17  0.092641
1 -0.018557 -2.818592e-17  0.015440
2  0.014028 -4.261311e-17 -0.011672
3  0.035209  0.000000e+00 -0.029296


In [166]:
X.values.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [37]:
def test_ttest_implementation(func):
    """`func` should be an implementation of ttest reweighting as defined above."""
    X = pd.DataFrame(np.array([
        [  4.,   4.,   2.,   0.],
        [  4.,  61.,   8.,  18.],
        [  2.,   8.,  10.,   0.],
        [  0.,  18.,   0.,   5.]]))    
    actual = np.array([
        [ 0.33056, -0.07689,  0.04321, -0.10532],
        [-0.07689,  0.03839, -0.10874,  0.07574],
        [ 0.04321, -0.10874,  0.36111, -0.14894],
        [-0.10532,  0.07574, -0.14894,  0.05767]])    
    predicted = func(X)
    assert np.array_equal(predicted.round(5), actual)

In [98]:
test_ttest_implementation(ttest_reweight)

Third, apply your implementation to the matrix stored in `imdb_window5-scaled.csv.gz`.

__To submit__: the cell value for the row labeled _superb_ and the column labeled _movie_.

(The goal here is really to obtain a working implementation of $\textbf{ttest}$. It could be an ingredient in a winning bake-off entry!)

In [54]:
data_home = 'vsmdata'
imdb5 = pd.read_csv(
    os.path.join(data_home, 'imdb_window5-scaled.csv.gz'), index_col=0)

In [100]:
imdb5_t = ttest_reweight(imdb5)

In [118]:
imdb5_t['superb']['movie'], imdb5_t['movie']['superb']

(-0.0008795624226660333, -0.0008427614547882469)

In [161]:
from itertools import islice
for word1 in imdb5_t:
    #print(word1)
    largest = imdb5_t[word1].nlargest(5)
    for i,val in enumerate(largest):
        word2 = largest.index[i]
        if word1 != word2 and val > 0.02:
            print(word1, 'and', word2)
            print(j, '\n')
    #for j in islice(imdb5_t[i]):
    #   print(imdb5_t[i][j])

* and SPOILER
0.000595357919454039 

** and SPOILERS
0.000595357919454039 

*** and SPOILERS
0.000595357919454039 

/ and >
0.000595357919454039 

> and /
0.000595357919454039 

I and saw
0.000595357919454039 

SPOILERS and ***
0.000595357919454039 

SPOILERS and **
0.000595357919454039 

[ and ]
0.000595357919454039 

[ and SPOILERS
0.000595357919454039 

citizen and kane
0.000595357919454039 

harry and potter
0.000595357919454039 

jackie and chan
0.000595357919454039 

mel and gibson
0.000595357919454039 

pearl and harbor
0.000595357919454039 

tim and burton
0.000595357919454039 

warning and spoilers
0.000595357919454039 

woody and allen
0.000595357919454039 



## Questions 4–6: Reweighting and co-occurrence frequency [3 points]

We've seen that raw count matrices encode a lot of frequency information. This is not necessarily all bad (stronger words like _superb_ will be rarer than weak ones like _good_ in part because of their more specialized semantics), but we do hope that our reweighting schemes will get us away from these relatively mundane associations. Thus, for any reweighting scheme, we should ask about its correlation with the raw co-occurrence counts.

Your task: using [scipy.stats.pearsonr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), calculate the Pearson correlation coefficient between the raw count values of `imdb5` as loaded in the previous question and the values obtained from applying PMI and Positive PMI to this matrix, and from reweighting each row by its length norm (as defined in the first noteboook for this unit; `vsm.length_norm`). Note: `X.values.ravel()` will give you the vector of values in the `pd.DataFrame` instance `X`.

__To submit:__

1. Correlation coefficient for the PMI comparison.
1. Correlation coefficient for the Positive PMI comparison.
1. Correlation coefficient for the length-norm comparison.

(The hope is that seeing these values will give you a better sense for how these reweighting schemes compare to the input count matrices.)

In [168]:
imdb5_ppmi   = vsm.pmi(imdb5)
imdb5_pmi    = vsm.pmi(imdb5, positive=False)
imdb5_normed = imdb5.apply(vsm.length_norm, axis=1)

In [176]:
reweightings = (('PMI', imdb5_pmi), ('PPMI', imdb5_ppmi), ('Length norm', imdb5_normed) )
raw = imdb5.values.ravel()
for x,y in reweightings:
    #print(x,len(y))
    print('Pearson coefficient for counts vs ', x, 'is: ', pearsonr(raw, y.values.ravel())[0])

Pearson coefficient for counts vs  PMI is:  0.007653071529112714
Pearson coefficient for counts vs  PPMI is:  0.04498063308409067
Pearson coefficient for counts vs  Length norm is:  0.1221866233067643


## Question 7: Meeting the GloVe objective [1 point]

We saw that GloVe can be thought of as seeking vectors whose dot products are proportional to their PMI values. How close does GloVe come to this in practice? This question asks you to conduct a simple empirical assessment of that: 

1. Load the matrix stored as `imdb_window5-scaled.csv.gz` in the data distribution. Call this `imdb5`.
2. Reweight `imdb5` with Positive PMI.
3. Run GloVe on `imdb5` for 10 iterations, learning vectors of dimension 20 (`n=20`). Definitely use the implementation in the `mittens` package, not in `vsm.glove`, else this will take way too long. Except for `max_iter` and `n`, use all the default parameters.
4. Report the correlation between the cell values in the PMI and GloVe versions. For this, you can include all 0 values (even though GloVe ignores them). Use `pearsonr` as above.

In [204]:
glove_model = GloVe(max_iter=500, n=100)

imdb5_glv = glove_model.fit(imdb5_ppmi.values)

imdb5_glv = pd.DataFrame(imdb5_glv, index=imdb5.index)

dispersion = imdb5_glv.dot(imdb5_glv.transpose())

Iteration 500: loss: 47.85442352294922

In [221]:
raw = imdb5_pmi.values.ravel()

x, y = 'GloVe', dispersion
print('Pearson coefficient for counts vs ', x, 'is: ', pearsonr(raw, y.values.ravel())[0])

Pearson coefficient for counts vs  GloVe is:  0.010589410809805126


In [223]:
glove_model_small = GloVe(max_iter=10, n=20)

imdb5_glv_small = glove_model_small.fit(imdb5_ppmi.values)

imdb5_glv_small = pd.DataFrame(imdb5_glv_small, index=imdb5.index)

dispersion_small = imdb5_glv_small.dot(imdb5_glv_small.transpose())

raw = imdb5_pmi.values.ravel()
x, y = 'GloVe', dispersion_small
print('Pearson coefficient for counts vs ', x, 'is: ', pearsonr(raw, y.values.ravel())[0])

Iteration 10: loss: 6206.59619140625

Pearson coefficient for counts vs  GloVe is:  0.011290424477873229


In [224]:
raw = imdb5_pmi.values.ravel()

x, y = 'GloVe', dispersion_small
print('Pearson coefficient for counts vs ', x, 'is: ', pearsonr(raw, y.values.ravel())[0])

Pearson coefficient for counts vs  GloVe is:  0.011290424477873229


In [225]:
raw = imdb5_ppmi.values.ravel()

x, y = 'GloVe', dispersion_small
print('Pearson coefficient for counts vs ', x, 'is: ', pearsonr(raw, y.values.ravel())[0])

Pearson coefficient for counts vs  GloVe is:  0.0758144041311364


## Question 8: Expressive eloooongation [2 points]

One of the goals of subword modeling is to capture out-of-vocabulary (OOV) words. This is particularly important for __expressive elogations__ like _coooooool_ and _booriiiing_. Because the amount of elongation is highly variable, we're unlikely to have good representations for such words. How does [our simple approach to subword modeling](vsm_01_distributional.ipynb#Subword-information) do with these phenomena?

__Your task:__

* Use `vsm.ngram_vsm` to create a 4-gram character-level VSM from the matrix in `imdb_window20-flat.csv.gz`.

* Using `character_level_rep` from the notebook for representing words in this space, calculate the cosine distances for pair `cool` and `cooooool`.

__To submit__: the cosine distance  between `cool` and `cooooool`

(Of course, the broader question we want to answer is whether these words are being modeled as similar, which is a more subjective, comparative question. It does depend on these distance calculations, though.)

In [207]:
imdb20 = pd.read_csv(
    os.path.join(data_home, 'imdb_window20-flat.csv.gz'), index_col=0)

In [208]:
imdb20_ngrams = vsm.ngram_vsm(imdb20, n=4)

In [210]:
def character_level_rep(word, cf, n=4):
    ngrams = vsm.get_character_ngrams(word, n)
    ngrams = [n for n in ngrams if n in cf.index]    
    reps = cf.loc[ngrams].values
    return reps.sum(axis=0)    

In [219]:
cool = character_level_rep("cool", imdb20_ngrams)
cooooool = character_level_rep("cooooool", imdb20_ngrams)
uncool = character_level_rep("uncool", imdb20_ngrams)
refrigerator = character_level_rep("refrigerator", imdb20_ngrams)
plug = character_level_rep("plug", imdb20_ngrams)
sun = character_level_rep("sun", imdb20_ngrams)



print(vsm.cosine(cool, cooooool))
print(vsm.cosine(cool, uncool))
print(vsm.cosine(cool, refrigerator))
print(vsm.cosine(cool, plug))
print(vsm.cosine(cool, sun))

0.0006569654844812423
0.0015116382525530714
0.031710700835408834
0.04945948672621814
0.048308110456579456
